In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from sklearn import svm

In [ ]:
#pobranie danych
df = pd.read_csv("https://raw.githubusercontent.com/mini-pw/2021L-WUM/main/Prace_domowe/Praca_domowa3/australia.csv")
df.head()

In [ ]:
df.hist(figsize=(12,16))
plt.show()

Dla pewności sprawdziłem rozkłady wszystkich zmiennych. Wydają się wyglądać w porządku, bez żadnych wyraźnych anomalii. Dodatkowo widać tu, że zmienna celu nie jest zbalansowana - mamy więcej zer.

In [ ]:
X = df.drop("RainTomorrow", axis=1)
y = df[["RainTomorrow"]]

Żeby sprawdzić, czy `X` i `y` dobrze się przycięły można wyświetlić kilka pierwszych wierszy i sprawdzić, czy ramki mają odpowiedni kształt.

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
y.describe()

Widać tutaj, że zbiór danych jest raczej przekrzywiony w stronę zera - mamy dysproporcję klas. Więcej wierszy pochodzi z dni, gdy nie padał deszcz. Wypada użyć argumentu `stratify` przy podziale, aby zachować odpowiednie proporcje klas.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2)

Pierwszym modelem, z którego skorzystam jest regresja logistyczna. W tym przypadku ustawiłem następujące hiperparametry:
- `penalty = "l2"` - ustawienie regularyzacji na l2
- `C = 1.2` - ustawienie wagi regularyzacji na trochę łagodniejszą niż domyślna
- `max_iter = 1000` - zwiększenie domyślnej maksymalnej liczby iteracji tego modelu

In [ ]:
logit = LogisticRegression(penalty="l2", C=1.2, max_iter=1000, random_state=42)
logit.fit(X_train, y_train)

logit_pred = logit.predict(X_test)

acc_logit = accuracy_score(y_test, logit_pred)
rec_logit = recall_score(y_test, logit_pred)
f1_logit = f1_score(y_test, logit_pred)

print("Accuracy: {:.3f}\nRecall: {:.3f}\nF1-score: {:.3f}".format(acc_logit, rec_logit, f1_logit))

Kolejnym modelem jest las losowy. Parametry, które tu ustawiłem to:
- `n_estimators=20` - liczba drzew, z których korzysta ten klasyfikator
- `max_depth=4` - największa wysosość pojedynczego drzewa
- `max_features=3` - ile najwięcej featerów może rozdzielać jedno drzewo

In [ ]:
rfc = RandomForestClassifier(n_estimators=20, max_depth=4,max_features=3, random_state=42)

rfc.fit(X_train, y_train)

rfc_pred = rfc.predict(X_test)

acc_rfc = accuracy_score(y_test, rfc_pred)
rec_rfc = recall_score(y_test, rfc_pred)
f1_rfc = f1_score(y_test, rfc_pred)

print("Accuracy: {:.3f}\nRecall: {:.3f}\nF1-score: {:.3f}".format(acc_rfc, rec_rfc, f1_rfc))

Kolejnym klasyfikatorem jest model opierający się na Stochastic Gradient Descent. Przy tym modelu wyspecyfikowałem następujące hiperparametry:
- `loss="hinge"` - określa funkcję straty
- `penalty="l2"` - regularyzacja l2

In [ ]:
sgd = SGDClassifier(loss="hinge", penalty="l2", max_iter=1000)
sgd.fit(X_train, y_train)

sgd_pred = sgd.predict(X_test)

acc_sgd = accuracy_score(y_test, sgd_pred)
rec_sgd = recall_score(y_test, sgd_pred)
f1_sgd = f1_score(y_test, sgd_pred)

print("Accuracy: {:.3f}\nRecall: {:.3f}\nF1-score: {:.3f}".format(acc_sgd, rec_sgd, f1_sgd))


Ostatnim modelem jest klasyfikator działający na SVM, zastosowałem w nim kernel `linear` - linowy.

In [ ]:
svc = svm.SVC(kernel="linear", random_state=42)
svc.fit(X_train, y_train)
svc_pred = svc.predict(X_test)

acc_svc = accuracy_score(y_test, svc_pred)
rec_svc = recall_score(y_test, svc_pred)
f1_svc = f1_score(y_test, svc_pred)

print("Accuracy: {:.3f}\nRecall: {:.3f}\nF1-score: {:.3f}".format(acc_svc, rec_svc, f1_svc))

In [ ]:
res_arr = [
    [acc_logit, rec_logit, f1_logit],
    [acc_rfc, rec_rfc, f1_rfc],
    [acc_sgd, rec_sgd, f1_sgd], 
    [acc_svc, rec_svc, f1_svc]
]

results = pd.DataFrame(res_arr,columns = ["Accuracy", "Recall", "F1-score"], index=["Logistic Regression", "Random Forest", "Stochastic Gradient Descent", "SVM Classifier"])

results

In [ ]:
fig, axs = plt.subplots(1,3, figsize=(18, 10))
sns.barplot(data=results, y="Accuracy",x=results.index,ax=axs[0])
sns.barplot(data=results, y="Recall",x=results.index,ax=axs[1])
sns.barplot(data=results, y="F1-score",x=results.index,ax=axs[2])
axs[0].tick_params(axis="x", rotation=90) 
axs[1].tick_params(axis="x", rotation=90) 
axs[2].tick_params(axis="x", rotation=90) 
axs[0].set_ylim([0,1]) 
axs[1].set_ylim([0,1]) 
axs[2].set_ylim([0,1]) 

plt.show()


Wybór najlepszego spośród tych modeli nie jest jednoznaczny jednak na pierwszy rzut oka SVM wydaje się być najlepszy, ewentualnie regresja logistyczna. Wybór zależy jednak również od tego, co jest dla nas najbardziej wartościowe. Jeżeli kosztowne dla nas będzie przewidzenie dnia suchego, podczas gdy faktycznie będzie padał deszcz, to powninniśmy optymalizować `recall`. Jeśli zaś sytuacja jest odwrotna: najwięcej kosztuje przewidzenie deszczu, gdy go nie będzie, to model powinniśmy optymalizować pod względem `precision`. Miarą, która daje nam najwięcej informacji o całokształcie modelu jest F1, więc jeśli wszystkie pomyłki są tak samo kosztowne, można podejmować decyzje na jej podstawie.